In [1]:
import cv2
import pytesseract
import os
import sqlite3
from collections import defaultdict
import re
import json

# Cấu hình đường dẫn Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng detailsau nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS detailsau (input_name TEXT, output_path TEXT)''')

# Hàm loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa
def clean_text(text):
    cleaned_text = re.sub(r'["\s;,\n\r\t]+', ' ', text)  # Loại bỏ \" , ; dấu cách thừa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Thay nhiều khoảng trắng thành một
    return cleaned_text

# Hàm xóa dữ liệu cũ trong bảng detailsau
def clear_old_data():
    cursor.execute('DELETE FROM detailsau')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng detailsau.")

# Hàm chính để xử lý OCR và gộp kết quả
def process_ocr_and_merge(input_dir):
    # Danh sách lưu kết quả tạm thời
    extracted_data = defaultdict(list)  # Dùng để gộp các trường cùng loại

    # Sử dụng OCR để trích xuất văn bản từ ảnh trong thư mục
    img_path = input_dir  # input_dir là một đường dẫn đến một tệp chứ không phải thư mục
    img_file = os.path.basename(img_path)
    
    img = cv2.imread(img_path)

    # Kiểm tra xem tên file có phải là img hay không, nếu có thì chỉ lưu đường dẫn ảnh
    field_name = os.path.splitext(img_file)[0].split("_")[0]  # Lấy tên trường từ tên file

    if field_name == "img":
        # Nếu là trường img, chỉ cần lưu đường dẫn ảnh vào cơ sở dữ liệu mà không thực hiện OCR
        cursor.execute('INSERT INTO detailsau (input_name, output_path) VALUES (?, ?)', 
                       (field_name, img_path))
        print(f"Field: {field_name} - Image path saved without OCR: {img_path}")
        return  # Kết thúc hàm nếu là trường img
    
    # Sử dụng OCR để trích xuất văn bản cho các trường khác
    text = pytesseract.image_to_string(img, lang="vie").strip()

    # Nếu văn bản là bytes, chuyển đổi thành chuỗi str (nếu cần)
    if isinstance(text, bytes):
        text = text.decode('utf-8')

    # Làm sạch văn bản (loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa)
    cleaned_text = clean_text(text)

    # Lưu kết quả vào danh sách
    extracted_data[field_name].append(cleaned_text)

    # Hiển thị ảnh và văn bản đã trích xuất (đã làm sạch)
    print(f"Field: {field_name}")
    print(f"Extracted Text: {cleaned_text}")
    print("-" * 40)

    # Lưu thông tin vào cơ sở dữ liệu detailsau
    for field, values in extracted_data.items():
        for value in values:
            cursor.execute('INSERT INTO detailsau (input_name, output_path) VALUES (?, ?)', 
                           (field, value))

    conn.commit()

# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Lấy đường dẫn ảnh từ cơ sở dữ liệu detectsau
cursor.execute('SELECT output_path FROM detectsau')
rows = cursor.fetchall()

# Duyệt qua từng đường dẫn ảnh lấy được từ cơ sở dữ liệu và gọi hàm xử lý OCR
for row in rows:
    input_dir = row[0]  # input_dir là đường dẫn đến tệp ảnh

    print(f"Processing image: {input_dir}")
    
    process_ocr_and_merge(input_dir)

# Gộp các trường giống nhau và lưu vào file JSON
final_data = defaultdict(list)

# Lấy tất cả dữ liệu từ bảng detailsau để gộp lại
cursor.execute('SELECT input_name, output_path FROM detailsau')
rows = cursor.fetchall()

for input_name, output_path in rows:
    if input_name.startswith("ddnd"):  
        final_data['ddnd'].append(output_path)
    else:
        final_data[input_name].append(output_path)

# Chuyển đổi danh sách thành chuỗi cho các trường gộp lại
for key in final_data.keys():
    final_data[key] = " ".join(final_data[key])

# Lưu kết quả vào file JSON
output_json_path = '../data/datasau.json'
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(final_data, json_file, ensure_ascii=False, indent=4)

print(f"\nData saved to {output_json_path}")

# Đóng kết nối cơ sở dữ liệu
conn.close()


Đã xóa dữ liệu cũ trong bảng detailsau.
Processing image: ../data/img/cropsau/tg_1.jpg
Field: tg
Extracted Text: 24/11/2022
----------------------------------------
Processing image: ../data/img/cropsau/ddnd_1.jpg
Field: ddnd
Extracted Text: NỘI mUỐỔi C CH đưới mép trá
----------------------------------------

Data saved to ../data/datasau.json
